# Pymarc Patterns

[Pymarc Documentation](https://pymarc.readthedocs.io/en/latest/)

This notebook covers common patterns for working with MARC records in Python. It will start with the very basics like getting fields and move into more complex examples.

The example records come from [Harvard's bibliographic records](https://archive.org/download/harvard_bibliographic_metadata) on the Internet Archive.

## Reading Records from Files

Use the `MARCReader` class to read records from a file, it accepts an open file handle and returns an iterator of `Record` objects. Make sure each record is not `None` when iterating.

In [14]:
from pymarc import MARCReader

with open('assets/example.mrc', 'rb') as fh:
    reader = MARCReader(fh)
    for record in reader:
        if record:
            print("Got record:", record.title.rstrip(' /'))
            # save this global variable for use in later code blocks
            global venetian
            venetian = record
        else:
            print("No record found.")

Got record: Photographs of Venetian villas


Note that the file is opened in read-binary mode (`rb`). Read mode is sufficient because we are not modifying the file. We use binary mode because Pymarc will handle decoding strings in records, we don't want Python to do it. Try deleting the `b` in `rb`—what happens? What would happen if we didn't have an `if record` condition?

There are several gotchas you can run into with encoding issues. The [`MARCReader`](https://pymarc.readthedocs.io/en/latest/#pymarc.reader.MARCReader) class has a `to_unicode` parameter to return UTF-8 strings as well as a `force_utf8` parameter which coerces the data to UTF-8 (useful if you have records with inaccurate encodings). These parameters seemed to be more commonly needed under Python 2 where it was more work to manage string encodings.



## Simple Ways to View Record Data

Pymarc comes with convenience properties for accessing common MARC fields on a record:

- `record.title`
- `record.author`
- `record.isbn`
- `record.publisher`
- `record.pubyear`

Also, if you print a record, its string representation is the "mnemonic marc" format, which is a human-readable version of the MARC data.

In [18]:
print('Title:', venetian.title)
print('Author:', venetian.author)
print('ISBN:', venetian.isbn)
print('Publisher:', venetian.publisher)
print('Year:', venetian.pubyear)
print(venetian)

Title: Photographs of Venetian villas /
Author: None
ISBN: None
Publisher: The Institute,
Year: 1954.
=LDR  00774nam a22002057u 4500
=001  000000010-8
=005  20020606090541.3
=008  821202|1954\\\\|||||||\\||||\|0||||eng|d
=035  0\$aocm78684367
=245  00$aPhotographs of Venetian villas /$cRoyal Institute of British Architects ; detailed information compiled by Giuseppe Mazzotti.
=246  3\$aVenetian villas.
=260  0\$aLondon, England :$bThe Institute,$c1954.
=300  \\$a39 p. ;$c21 cm.
=500  \\$aCover title : Venetian villas.
=500  \\$aCatalogue of an exhibition held at Royal Institute of British Architects, Feb. 25-Mar. 27, 1954.
=650  \0$aArchitecture, Domestic$zItaly$zVenice.
=700  1\$aMazzotti, Giuseppe.
=710  2\$aRoyal Institute of British Architects.
=988  \\$a20020608
=906  \\$0MH



As you can see, the properties are set to `None` if they do not exist, like the ISBN in our example.

For serious work with records, **you should not use these convenience properties**. They only return the first instance of a field and certain subfields. They are useful for quick peaks at data, but not functional for most purposes. Instead, we will typically use the `get_fields()` method and iterate over all existing fields. There are also `record.subjects` and `record.notes` properties which return lists of actual `Field` objects and these are more useful.

## Modifying & Writing Records

The basic steps to modify MARC records with Pymarc are:

- Read the records in with `MARCReader`
- Modify them in place—you can simply assign values to fields and subfields
- Write the records out with `MARCWriter`

Below, we prefix the example record's title field with "Great".

In [21]:
from pymarc import MARCReader, MARCWriter

with open('assets/example.mrc', 'rb') as fh:
    reader = MARCReader(fh)
    with open('assets/great.mrc', 'wb') as out:
        writer = MARCWriter(out)
        for record in reader:
            if record:
                record["245"]["a"] = f'Great {record["245"]["a"]}'
                writer.write(record)
                print(record.title.rstrip(' /'))

Great Photographs of Venetian villas
